<a href="https://colab.research.google.com/github/dla9944/God_damn_deeplearning/blob/master/RNN_IMDB_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_T1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.10.1 --user -q

In [ ]:
!pip show torch torchtext

Name: torch
Version: 1.9.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /root/.local/lib/python3.8/site-packages
Requires: typing-extensions
Required-by: fastai, torchaudio, torchtext, torchvision
---
Name: torchtext
Version: 0.10.1
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /root/.local/lib/python3.8/site-packages
Requires: numpy, requests, torch, tqdm
Required-by: 


In [ ]:
# 필수 라이브러리
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets

In [ ]:
# 패러미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(f'running in {DEVICE}')

running in cuda


In [ ]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True) 

In [ ]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:09<00:00, 9.16MB/s]


In [ ]:
print(f"훈련셋 : {len(trainset)}")
print(f"시험셋 : {len(testset)}")

훈련셋 : 25000
시험셋 : 25000


In [ ]:
trainset[0].text[:10] 

['if',
 'you',
 'want',
 'to',
 'learn',
 'something',
 'about',
 'the',
 'spanish',
 'civil']

In [ ]:
# 단어 사전
TEXT.build_vocab(trainset, min_freq=5) 
LABEL.build_vocab(trainset)

In [ ]:
TEXT.vocab.freqs.most_common(100) 

[('the', 322198),
 ('a', 159953),
 ('and', 158572),
 ('of', 144462),
 ('to', 133967),
 ('is', 104171),
 ('in', 90527),
 ('i', 70480),
 ('this', 69714),
 ('that', 66292),
 ('it', 65505),
 ('/><br', 50935),
 ('was', 47024),
 ('as', 45102),
 ('for', 42843),
 ('with', 42729),
 ('but', 39764),
 ('on', 31619),
 ('movie', 30887),
 ('his', 29059),
 ('are', 28743),
 ('not', 28597),
 ('film', 27777),
 ('you', 27564),
 ('have', 27344),
 ('he', 26177),
 ('be', 25691),
 ('at', 22731),
 ('one', 22480),
 ('by', 21976),
 ('an', 21240),
 ('they', 20624),
 ('from', 19934),
 ('all', 19740),
 ('who', 19407),
 ('like', 18779),
 ('so', 18099),
 ('just', 17309),
 ('or', 16769),
 ('has', 16570),
 ('her', 16540),
 ('about', 16486),
 ("it's", 15970),
 ('some', 15280),
 ('if', 15189),
 ('out', 14510),
 ('what', 14055),
 ('very', 13633),
 ('when', 13609),
 ('more', 13170),
 ('there', 13094),
 ('she', 12234),
 ('would', 12027),
 ('even', 12010),
 ('good', 11926),
 ('my', 11766),
 ('only', 11566),
 ('their', 11317)

In [ ]:
LABEL.vocab.freqs

Counter({'pos': 12500, 'neg': 12500})

In [ ]:
trainset, valset = trainset.split(split_ratio=0.8)

In [ ]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (trainset, valset, testset),
    batch_size=BATCH_SIZE,
    shuffle=True, repeat=False
)


In [ ]:
vocab_size = len(TEXT.vocab)
n_classes = 2

In [ ]:
print(f"[학습셋] {len(trainset)} / [검증셋] {len(valset)} / 테스트셋 {len(testset)} / [단어수] {vocab_size} / [클래스(라벨)] {n_classes}")

[학습셋] 20000 / [검증셋] 5000 / 테스트셋 25000 / [단어수] 46159 / [클래스(라벨)] 2


In [ ]:
class BasicGRU(nn.Module):
    def __init__(self,
                 n_layers, 
                 hidden_dim,
                 n_vocab, 
                 embed_dim,
                 n_classes, 
                 dropout_p=0.2
                 ):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers, batch_first=True)

        self.out = nn.Linear(hidden_dim, n_classes)
    
    def forward(self, x): 
        x = self.embed(x) 
        h_0 = self._init_state(batch_size=x.size(0)) 
        x, _ = self.gru(x, h_0) 
        self.dropout(h_t)
        logit = self.out(h_t)
        return logit

    def _init_state(self, batch_size=1):

        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()


In [ ]:
# 학습함수
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):

        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)

        y.data.sub_(1) 
        optimizer.zero_grad()

        logit = model(x) 

        loss = F.cross_entropy(logit, y) 
        loss.backward() 
        optimizer.step() 

In [ ]:
# 평가함수
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)

        logit = model(x)

        loss = F.cross_entropy(logit, y, reduction='sum') 
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = corrects / size * 100 
    return avg_loss, avg_accuracy

In [ ]:
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:

best_val_loss = None
for epoch in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print(f"[에포크: {epoch}] 검증 오차:{val_loss:5.2f} | 검증 정확도:{val_accuracy:5.2f}")

    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot") 
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss # 오차 갱신

[에포크: 1] 검증 오차: 0.69 | 검증 정확도:49.68
[에포크: 2] 검증 오차: 0.69 | 검증 정확도:50.70
[에포크: 3] 검증 오차: 0.62 | 검증 정확도:67.32
[에포크: 4] 검증 오차: 0.39 | 검증 정확도:83.80
[에포크: 5] 검증 오차: 0.34 | 검증 정확도:86.20
[에포크: 6] 검증 오차: 0.35 | 검증 정확도:86.34
[에포크: 7] 검증 오차: 0.40 | 검증 정확도:86.70
[에포크: 8] 검증 오차: 0.45 | 검증 정확도:85.62
[에포크: 9] 검증 오차: 0.44 | 검증 정확도:85.58
[에포크: 10] 검증 오차: 0.50 | 검증 정확도:85.54


In [ ]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.35 | 테스트 정확도: 85.35
